Start

In [ ]:
import torch
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd
import numpy as np
from tqdm import tqdm
from loguru import logger

In [ ]:
tag = "nlptown/bert-base-multilingual-uncased-sentiment"

In [ ]:
from transformers import pipeline

# models are downloaded to ~/.cache/huggingface/hub.
# you might want to clean up that location after you are done with the models
model = pipeline(
    model="lxyuan/distilbert-base-multilingual-cased-sentiments-student",
    top_k=None,
    truncation=True,  # Truncate long inputs automatically
    max_length=512,
)

# english
model("I love this movie and i would watch it again and again!")

In [ ]:
import tomllib

configfile = Path("../config.toml").resolve()
with configfile.open("rb") as f:
    config = tomllib.load(f)
datafile = (Path("..") / Path(config["processed"]) / config["current"]).resolve()
if not datafile.exists():
    logger.warning(
        "Datafile does not exist. First run src/preprocess.py, and check the timestamp!"
    )
df = pd.read_parquet(datafile)
df.head()

In [ ]:
root = Path("..").resolve()
processed = root / Path(config["processed"])
raw = root / Path(config["raw"])
datafile = processed / config["current"]
role_file = raw / config["role_file"]
player_roles = pd.read_json(role_file, encoding = "latin")
player_roles.head()

In [ ]:
merged_df = pd.merge(df, player_roles, left_on='author', right_on='Author')
merged_df = merged_df.drop(columns=['Author'])
merged_df.head()

In [ ]:
from dataclasses import dataclass


@dataclass
class Position:
    name: str
    alltext: str
    chunked: list[str]
    sentiment: dict = None


# extract the data from the dataframe
datadict = {}
for i, row in tqdm(merged_df.iterrows(), total=len(df)):
    position = row["Position"]
    message = row["message"]
    if position not in datadict:
        datadict[position] = Position(name=position, alltext=message, chunked=[])
    else:
        datadict[position].alltext += message


def split_into_chunks(text, chunk_size=512):
    # Split the text into words
    words = text.split()
    # Create chunks of the specified size
    chunks = [
        " ".join(words[i : i + chunk_size]) for i in range(0, len(words), chunk_size)
    ]
    return chunks


# For every author, try to split their text into chunks of 512 tokens
for author in datadict:
    # Get the combined text for the author
    text = datadict[author].alltext
    # Split the text into chunks
    # we want 512 tokens, so lets guess about 50%
    datadict[author].chunked = split_into_chunks(text, chunk_size=400)

In [ ]:
def fill_sentiment(model, author: Position) -> Position:
    sentiment = [model(chunk)[0] for chunk in author.chunked]
    result = {}
    for item in sentiment:
        for entry in item:
            # Get the label and the score
            label = entry["label"]
            score = round(entry["score"], 4)  # Round the score to 4 decimal places

            # Append the score to the corresponding list in the dictionary
            if label not in result:
                result[label] = (
                    []
                )  # Initialize a list if the label is not in the dictionary
            result[label].append(score)
    author.sentiment = result
    return author


for key, item in tqdm(datadict.items(), total=len(datadict)):
    try:
        datadict[key] = fill_sentiment(model, item)
    except Exception as e:
        logger.warning(f"Failed to process message {key}")
        logger.warning(f"Error: {e}")

In [ ]:
# calculate collective positive average
total = []
for item in datadict.values():
    total.extend(item.sentiment["positive"])
avg = np.mean(total)

In [ ]:
long_format = []
mood = "negative"
for key, item in datadict.items():
    # only keep authors with more than 10 chunks of data
    if len(item.sentiment[mood]) < 5:
        continue
    for val in item.sentiment[mood]:
        long_format.append({"name": key, mood: val})
long_df = pd.DataFrame(long_format).sort_values(by=mood, ascending=False)
long_df.head()
sns.stripplot(x="name", y=mood, data=long_df, jitter=True, alpha=0.5)
plt.axhline(avg, color="red", linestyle="--")
plt.xticks(rotation=90, ha="center")
plt.title(f"Sentiment Analysis: {mood} sentiment")